In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

# Set random seed for reproducibility
torch.manual_seed(42)

In [3]:
# Define the RNN model
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, h0, c0):
        out, (hidden, cell) = self.rnn(x, (h0, c0))
        out = self.fc(out)  # Apply linear layer to all time steps
        return out, (hidden, cell)

# Hyperparameters
input_size = 1       # Each input is a single feature
hidden_size = 16     # Size of hidden state
output_size = 1      # Predicting a single value
seq_len = 10         # Length of each sequence
batch_size = 4       # Number of sequences in a batch
num_epochs = 200      # Number of training epochs
learning_rate = 0.01

In [4]:
# Create synthetic data
# Example: input sequences are sinusoids, and targets are shifted versions
x_train = torch.sin(torch.linspace(0, 2 * 3.1416, seq_len * batch_size).view(batch_size, seq_len, 1))
y_train = torch.cos(torch.linspace(0, 2 * 3.1416, seq_len * batch_size).view(batch_size, seq_len, 1))

# Initialize the model, loss function, and optimizer
model = SimpleRNN(input_size, hidden_size, output_size)
criterion = nn.MSELoss()  # Mean Squared Error Loss
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
in_data = torch.randn((batch_size, seq_len, input_size))
h0 = torch.zeros((1, batch_size, hidden_size))
c0 = torch.zeros((1, batch_size, hidden_size))
out, (hidden, cell) = model(in_data, h0, c0)

print(f'Output shape {out.shape}, hidden state shape: {hidden.shape}')

Output shape torch.Size([4, 10, 1]), hidden state shape: torch.Size([1, 4, 16])


In [6]:
# Training loop
for epoch in range(num_epochs):
    # Initialize the hidden state
    hidden = torch.zeros(1, batch_size, hidden_size)  # Shape: (num_layers, batch_size, hidden_size)
    cell = torch.zeros(1, batch_size, hidden_size)

    # Forward pass
    outputs, (hidden, cell) = model(x_train, hidden, cell)
    loss = criterion(outputs, y_train)  # Compute loss for all time steps
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print loss for every epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/200], Loss: 0.5224
Epoch [2/200], Loss: 0.5152
Epoch [3/200], Loss: 0.5112
Epoch [4/200], Loss: 0.5067
Epoch [5/200], Loss: 0.5016
Epoch [6/200], Loss: 0.4964
Epoch [7/200], Loss: 0.4909
Epoch [8/200], Loss: 0.4840
Epoch [9/200], Loss: 0.4753
Epoch [10/200], Loss: 0.4651
Epoch [11/200], Loss: 0.4541
Epoch [12/200], Loss: 0.4430
Epoch [13/200], Loss: 0.4332
Epoch [14/200], Loss: 0.4256
Epoch [15/200], Loss: 0.4164
Epoch [16/200], Loss: 0.3996
Epoch [17/200], Loss: 0.3786
Epoch [18/200], Loss: 0.3664
Epoch [19/200], Loss: 0.3481
Epoch [20/200], Loss: 0.3366
Epoch [21/200], Loss: 0.3370
Epoch [22/200], Loss: 0.3387
Epoch [23/200], Loss: 0.3391
Epoch [24/200], Loss: 0.3386
Epoch [25/200], Loss: 0.3376
Epoch [26/200], Loss: 0.3356
Epoch [27/200], Loss: 0.3309
Epoch [28/200], Loss: 0.3213
Epoch [29/200], Loss: 0.3110
Epoch [30/200], Loss: 0.3186
Epoch [31/200], Loss: 0.3047
Epoch [32/200], Loss: 0.3050
Epoch [33/200], Loss: 0.3073
Epoch [34/200], Loss: 0.2990
Epoch [35/200], Loss: 0

In [7]:
# Test the model (inference)
with torch.no_grad():
    test_hidden = torch.zeros(1, batch_size, hidden_size)
    test_cell = torch.zeros(1, batch_size, hidden_size)
    test_outputs, _ = model(x_train, test_hidden, test_cell)
    print("\nSample output (first sequence):")
    print("Predicted:", test_outputs[0].squeeze().numpy())
    print("Target:   ", y_train[0].squeeze().numpy())


Sample output (first sequence):
Predicted: [-0.02943501  0.5567978   0.9707504   0.92432016  0.7900776   0.663672
  0.5660973   0.45813662  0.31116766  0.11457111]
Target:    [1.         0.9870502  0.9485362  0.8854555  0.7994419  0.69272304
 0.5680629  0.42869017 0.2782146  0.12053337]


In [8]:
import numpy as np

# Example input array of IDs
ids = np.array([1, 1, 1, 2, 2, 2, 2, 3, 3, 4, 4, 4, 5])
ids = np.repeat(ids[None, :], repeats=4, axis=0)
ids[2, :] = 1

# Length of False values to add after each change
false_length = 2
result = np.full(ids.shape, fill_value=True)

for i in range(ids.shape[0]):
    # Step 1: Find where changes occur
    change_indices = np.nonzero(np.diff(ids[i]) != 0)[0] + 1

    # Step 2: Create an output array
    # Initialize the result as an empty array
    if change_indices.size > 0:
        indices = np.array([np.arange(idx, idx+false_length) for idx in change_indices])
        indices = np.minimum(indices, ids[i].shape[0]-1)
        result[i, indices] = False

print("Original IDs:", ids)
# print("Change indices:", change_indices)
print("Resulting Boolean Array:", result)


Original IDs: [[1 1 1 2 2 2 2 3 3 4 4 4 5]
 [1 1 1 2 2 2 2 3 3 4 4 4 5]
 [1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 2 2 2 2 3 3 4 4 4 5]]
Resulting Boolean Array: [[ True  True  True False False  True  True False False False False  True
  False]
 [ True  True  True False False  True  True False False False False  True
  False]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True]
 [ True  True  True False False  True  True False False False False  True
  False]]
